<a href="https://colab.research.google.com/github/campagnucci/dados-abertos-gov-br/blob/master/API_Orcamento_Coda2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gastos públicos federais via API - Despesas e contratos 



Workshop no Coda.Br 2019

*Fernanda Campagnucci*

Diretora-Executiva da Open Knowledge Brasil 

Duração: 1h15

Tópicos:

1. **API x Bases de Dados**
- Quando usar?
- Onde encontrar

2. **Planejamento e Orçamento Público**
- Princípios do orçamento 
- Classificações da Despesa
- Execução Orçamentária e estágios da despesa 

3. **Acessando a API**

- Despesas
- Contratos

4. **Mais referências**

## 1. API x Bases de Dados
---

### Quando usar?
Quem não tem fluência em programação (como eu!), pode precisar de uma certa curva de aprendizagem para usar a API. Em alguns casos, porém, você pode ganhar muito tempo se optar pela API, mesmo que demore mais para criar seu código pela primeira vez. *Atenção: essa comparação não é geral, se aplica especificamente ao Portal da Transparência do Governo Federal e à maneira como os dados estão disponibilizados hoje*. 

| Situação                       |   API           | Base de dados  |
| :-------------                 | :-------------  | :------------- |
| Preciso de uma série histórica grande| +               | +++++          |
| Não faço pauta desse tema com frequência | ++               | +++++|
| Quero monitorar um assunto/cnpj/órgão etc | +++++               | +          |
| Quero conectar em um site ou visualização | ++++               | ++      |
| *Que outras situações ocorrem a vocês?* |               |      |
 
 


### Onde encontrar?

Para conhecer os dados disponíveis, sistema de origem, órgão responsável, periodicidade e data de atualização do Portal (download e API):

http://transparencia.gov.br/origem-dos-dados

- **API - Application Programming Interface**

http://www.transparencia.gov.br/swagger-ui.html

Swagger: Ferramenta que documenta e permite explorar o conteúdo 

- **Bases para download**

http://transparencia.gov.br/download-de-dados

https://brasil.io/dataset/gastos-diretos/gastos (coletado e tratado pelo @Turicas)


## 2. Planejamento e orçamento público 
---


![planejamento e orçamento](http://umdadoamais.com/wp-content/uploads/2019/11/tutorial_orcamento1-e1574301837420.png)




### Princípios do Orçamento

O orçamento público deve observar um conjunto de princípios presentes na Constituição e nas leis que regem o processo orçamentário no Brasil, em especial a Lei nº 4.320/1964 e a Lei nº 101/2000:

![alt text](http://umdadoamais.com/wp-content/uploads/2019/11/tutorial_orcamento2.png)

O orçamento é *autorizativo*, não obrigatório

### Classificações de Despesa

![alt text](http://umdadoamais.com/wp-content/uploads/2019/11/tutorial_orcamento3.png)




![alt text](http://umdadoamais.com/wp-content/uploads/2019/11/Screen-Shot-2019-11-23-at-00.59.24.png)

Exercício no Swagger: http://www.transparencia.gov.br/swagger-ui.html#!/Despesas32P250blicas/despesasPorFuncaoUsingGET

![alt text](http://umdadoamais.com/wp-content/uploads/2019/11/tutorial_orcamento6.png)

![alt text](http://umdadoamais.com/wp-content/uploads/2019/11/tutorial_orcamento7.png)

E se fosse buscar esse dado pela base para download? 
- Baixar cada mês e juntar as bases
- Filtrar o campo de ação
- Somar registros de empenhado, liquidado, pago

Vantagem: é desagregado


## 3. Acessando a API

---



In [0]:
import pandas as pd
import requests
import json
import numpy as np

### Despesas 

Exercício 1. Coletar uma série histórica de despesas do Ministério do Meio Ambiente 

In [0]:
serie_orgao = []

for ano in range(2014, 2020):

    url = f'https://api.portaldatransparencia.gov.br/api-de-dados/despesas/por-orgao?ano={ano}&orgaoSuperior=44000&pagina=1'
    # gerada a partir da consulta por órgão no swagger
    
    r_orgao = requests.get(url, verify=True).json()

    serie_orgao = serie_orgao + r_orgao

In [3]:
df_serie_orgao = pd.DataFrame(serie_orgao)
df_serie_orgao.tail(5)

,ano,orgao,codigoOrgao,orgaoSuperior,codigoOrgaoSuperior,empenhado,liquidado,pago
31,2019,Instituto Brasileiro do Meio Ambiente e dos Re...,20701,Ministério do Meio Ambiente,44000,"1.419.953.161,70","1.120.111.516,08","1.111.155.041,62"
32,2019,Fundo Nacional do Meio Ambiente,44204,Ministério do Meio Ambiente,44000,"51.453,33","46.695,22","46.695,22"
33,2019,Instituto Chico Mendes de Conservação da Biodi...,44207,Ministério do Meio Ambiente,44000,"576.288.062,35","490.458.991,03","488.308.637,44"
34,2019,Fundo Nacional sobre Mudança no Clima,44902,Ministério do Meio Ambiente,44000,"350.719.518,00","0,00","0,00"
35,2019,Instituto de Pesquisas Jardim Botânico do Rio ...,44206,Ministério do Meio Ambiente,44000,"74.466.616,95","54.759.381,21","54.712.683,17"


Exercicio 1.2 - Agrupar por orgao e comparar ano a ano

In [4]:
df_por_orgao = pd.pivot_table(df_serie_orgao, 
                              values='liquidado',
                              index=['orgao'],
                              columns='ano',
                              aggfunc=np.sum).sort_values(by='orgao', ascending=False)
                            
df_por_orgao

ano,2014,2015,2016,2017,2018,2019
orgao,,,,,,
Ministério do Meio Ambiente - Unidades com vínculo direto,"290.361.669,73","320.092.573,08","292.733.441,47","291.294.400,47","229.293.617,08","143.621.004,33"
Instituto de Pesquisas Jardim Botânico do Rio de Janeiro,"59.148.259,04","61.390.469,20","64.956.162,41","68.872.033,31","70.106.689,12","54.759.381,21"
Instituto Chico Mendes de Conservação da Biodiversidade,"521.298.991,82","536.502.218,06","574.340.871,93","584.586.460,34","610.110.950,78","490.458.991,03"
Instituto Brasileiro do Meio Ambiente e dos Recursos Naturais Renováveis,"1.206.848.908,65","1.271.750.234,93","1.322.018.555,52","1.372.364.904,86","1.406.268.441,60","1.120.111.516,08"
Fundo Nacional sobre Mudança no Clima,"300.000,00","0,00","0,00","23.166.881,00","392.582.919,00","0,00"
Fundo Nacional do Meio Ambiente,"3.919.125,95","26.713,25","241.528,51","1.811.643,23","1.181.420,06","46.695,22"


Exercicio 2 - Coletar dados de execução orçamentária da subfunção controle ambiental (542)

In [0]:
ano=2019 #mudar de acordo com a necessidade
page=1 #requisito obrigatório da API
subfuncao=542 #controle ambiental

In [6]:
url = f'http://www.transparencia.gov.br/api-de-dados/despesas/por-funcional-programatica?ano={ano}&subfuncao={subfuncao}'
#gerada no Swagger na consulta funcional programática
url

'http://www.transparencia.gov.br/api-de-dados/despesas/por-funcional-programatica?ano=2019&subfuncao=542'

In [0]:
r_funcional = requests.get(url+f'&pagina={page}',
                           verify=True).json()

Mas... Precisamos fazer uma iteração para coletar todas as páginas

In [8]:
page = 1
todas_pag = []
todas_pag = todas_pag + r_funcional
tamanho = int(requests.get(url+f'&pagina={page}').headers['Content-Length'])
#verifica tamanho da resposta; 22 é padrão da resposta vazia, ou seja, não há conteúdo nessa página

while(tamanho>22):
    page = page +1
    print(page)  
    r_funcional = requests.get(url+f'&pagina={page}').json() 
    print(url+f'&pagina={page}')
    todas_pag = todas_pag + r_funcional
    tamanho = int(requests.get(url+f'&pagina={page}').headers['Content-Length'])
    print(tamanho)

2
http://www.transparencia.gov.br/api-de-dados/despesas/por-funcional-programatica?ano=2019&subfuncao=542&pagina=2
722
3
http://www.transparencia.gov.br/api-de-dados/despesas/por-funcional-programatica?ano=2019&subfuncao=542&pagina=3
22


In [0]:
df_funcional = pd.DataFrame(todas_pag)

In [29]:
df_funcional.head()

,ano,funcao,codigoFuncao,subfuncao,codigoSubfuncao,programa,codigoPrograma,acao,codigoAcao,empenhado,liquidado,pago
0,2019,Gestão ambiental,18,Controle ambiental,542,QUALIDADE AMBIENTAL,2083,LICENCIAMENTO AMBIENTAL FEDERAL,6925,"4.604.677,75","2.373.900,40","2.326.754,32"
1,2019,Gestão ambiental,18,Controle ambiental,542,MUDANCA DO CLIMA,2050,PREVENCAO E CONTROLE DE INCENDIOS FLORESTAIS N...,214M,"36.589.934,14","28.870.331,05","25.091.211,26"
2,2019,Defesa nacional,05,Controle ambiental,542,DEFESA NACIONAL,2058,IMPLANTACAO DO SISTEMA AMAZONIA - SAR,15P5,"1.368.364,36","298.364,36","298.364,36"
3,2019,Gestão ambiental,18,Controle ambiental,542,"FLORESTAS, PREVENCAO E CONTROLE DO DESMATAMENT...",2036,PREVENCAO E COMBATE A INCENDIOS FLORESTAIS EM ...,20MY,"0,00","0,00","0,00"
4,2019,Gestão ambiental,18,Controle ambiental,542,QUALIDADE AMBIENTAL,2083,AVALIACAO DE PERICULOSIDADE E CONTROLE DE PROD...,20WH,"1.516.189,89","843.027,57","770.291,75"


Exercicio 2.1 - Agrupar empenhos por programa e função

In [46]:
def limpa_empenho(linha):
  """retira os pontos e coloca vírgula no lugar de ponto para posterior conversão em número"""
  valor = linha['empenhado']
  valor = valor.replace('.', '')
  valor = valor.replace(',', '.')
  return valor

df_funcional['valor_empenhado'] = df_funcional.apply(limpa_empenho, axis =1)

df_funcional['valor_empenhado'] = df_funcional.valor_empenhado.astype('float64')  

df_por_programa = pd.pivot_table(df_funcional, 
                              values='valor_empenhado',
                              index=['programa'],
                              columns='funcao',
                              aggfunc=np.sum)
                          
df_por_programa

funcao,Ciência e Tecnologia,Defesa nacional,Energia,Gestão ambiental,Transporte
programa,,,,,
CONSERVACAO E USO SUSTENTAVEL DA BIODIVERSIDADE,NaN,NaN,NaN,14712569.18,NaN
DEFESA NACIONAL,NaN,62623616.9,NaN,NaN,NaN
"FLORESTAS, PREVENCAO E CONTROLE DO DESMATAMENTO E DOS INCEND",NaN,NaN,NaN,0.00,NaN
MUDANCA DO CLIMA,NaN,NaN,NaN,73837753.71,NaN
PETROLEO E GAS,NaN,NaN,1123316.0,NaN,NaN
POLITICA NUCLEAR,0.0,NaN,NaN,NaN,NaN
PROGRAMA DE GESTAO E MANUTENCAO DO MINISTERIO DA INFRAESTRUT,NaN,NaN,NaN,NaN,0.0
QUALIDADE AMBIENTAL,NaN,NaN,NaN,11141566.63,NaN
RESÍDUOS SÓLIDOS URBANOS,NaN,NaN,NaN,0.00,NaN


Exercicio 2.2 - Ranking de ações dentro de controle ambiental, empenhado x liquidado

In [48]:
def limpa_liquidado(linha):
  """retira os pontos e coloca vírgula no lugar de ponto para posterior conversão em número"""
  valor = linha['liquidado']
  valor = valor.replace('.', '')
  valor = valor.replace(',', '.')
  return valor

df_funcional['valor_liquidado'] = df_funcional.apply(limpa_liquidado, axis =1)

df_funcional['valor_liquidado'] = df_funcional.valor_liquidado.astype('float64') 

df_por_acao = pd.pivot_table(df_funcional, 
                              values=['valor_empenhado', 'valor_liquidado'],
                              index=['acao'],
                              aggfunc=np.sum).sort_values(by='valor_liquidado', ascending=False)
                            
df_por_acao

,valor_empenhado,valor_liquidado
acao,,
PREVENCAO E CONTROLE DE INCENDIOS FLORESTAIS NAS AREAS FEDERAIS PRIORITARIAS,73179868.28,57740662.10
MANUTENCAO DO SISTEMA DE PROTECAO DA AMAZONIA - SIPAM,59886888.18,33693359.54
GESTAO DO USO SUSTENTAVEL DA BIODIVERSIDADE,14712569.18,8607894.55
LICENCIAMENTO AMBIENTAL FEDERAL,4604677.75,2373900.40
MONITORAMENTO AMBIENTAL E GESTAO DA INFORMACAO SOBRE O MEIO AMBIENTE E EDUCACAO AMBIENTAL,4326970.48,1957206.86
"AVALIACAO DE PERICULOSIDADE E CONTROLE DE PRODUTOS, SUBSTANCIAS QUIMICAS E RESIDUOS PERIGOSOS",1516189.89,843027.57
IMPLANTACAO DO SISTEMA AMAZONIA - SAR,2736728.72,596728.72
ESTUDOS AMBIENTAIS DE AREAS SEDIMENTARES,1123316.00,547757.74
APOIO A IMPLEMENTACAO DE INSTRUMENTOS ESTRUTURANTES DA POLITICA NACIONAL DE RESIDUOS SOLIDOS,572723.67,239930.96


### Contratos

Exercicio 3. Verificar contratos do Ministério do Meio Ambiente

Primeiro, explorar campos de contrato no Swagger e gerar URL:

![alt text](http://umdadoamais.com/wp-content/uploads/2019/11/tutorial_orcamento8.png)

In [0]:
#parâmatros para a iteração
dia_inicio = '01'
mes_inicio = '01'
dia_fim = '31'
mes_fim = '12'
ano = 2018
page = 1

In [14]:
url = f'http://www.transparencia.gov.br/api-de-dados/contratos?dataInicial={dia_inicio}%2F{mes_inicio}%2F{ano}&dataFinal={dia_fim}%2F{mes_fim}%2F2019&codigoOrgao=44000'

r_contratos = requests.get(url+f'&pagina={page}',
                           verify=True).json()

page = 1
todas_pag = []
todas_pag = todas_pag + r_contratos
tamanho = int(requests.get(url+f'&pagina={page}').headers['Content-Length'])
#verifica tamanho da resposta; 22 é padrão da resposta vazia, ou seja, não há conteúdo nessa página

while(tamanho>22):
    page = page +1
    print(page)  
    r_contratos = requests.get(url+f'&pagina={page}').json() 
    print(url+f'&pagina={page}')
    todas_pag = todas_pag + r_contratos
    tamanho = int(requests.get(url+f'&pagina={page}').headers['Content-Length'])
    print(tamanho)


2
http://www.transparencia.gov.br/api-de-dados/contratos?dataInicial=01%2F01%2F2018&dataFinal=31%2F12%2F2019&codigoOrgao=44000&pagina=2
3265
3
http://www.transparencia.gov.br/api-de-dados/contratos?dataInicial=01%2F01%2F2018&dataFinal=31%2F12%2F2019&codigoOrgao=44000&pagina=3
22


In [15]:
df_contratos = pd.DataFrame(todas_pag)
df_contratos.head()

,id,dimCompra,situacaoContrato,modalidadeCompra,unidadeGestora,unidadeGestoraCompras,dataAssinatura,dataPublicacaoDOU,dataInicioVigencia,dataFimVigencia,fornecedor,valorInicialCompra,valorFinalCompra,contrato
0,38464503,"{'numero': '000112018', 'objeto': 'Objeto: Pre...",None,None,"{'codigo': '440001', 'nome': 'SUBSECRET.DE PLA...",None,18/12/2018,20/12/2018,18/12/2018,17/12/2019,"{'numeroInscricaoSocial': '', 'nome': 'DELL CO...",2496600.0,2496600.0,"{'numero': '222018', 'objeto': 'Objeto: Aquisi..."
1,40464821,"{'numero': '000022018', 'objeto': 'Objeto: Pre...",None,None,"{'codigo': '440001', 'nome': 'SUBSECRET.DE PLA...",None,21/05/2018,22/05/2018,21/05/2018,20/05/2019,"{'numeroInscricaoSocial': '', 'nome': 'GAP SER...",378945.0,378945.0,"{'numero': '72018', 'objeto': 'Objeto: Contrat..."
2,40665800,"{'numero': '000062018', 'objeto': 'Objeto: Pre...",None,None,"{'codigo': '440001', 'nome': 'SUBSECRET.DE PLA...",None,17/12/2018,18/12/2018,17/12/2018,16/12/2019,"{'numeroInscricaoSocial': '', 'nome': 'BLUE EY...",560195.0,560195.0,"{'numero': '212018', 'objeto': 'Objeto: Serviç..."
3,40863029,"{'numero': '000062018', 'objeto': 'Objeto: Pre...",None,None,"{'codigo': '440001', 'nome': 'SUBSECRET.DE PLA...",None,12/12/2018,13/12/2018,12/12/2018,11/12/2019,"{'numeroInscricaoSocial': '', 'nome': 'BLUE EY...",598965.0,598965.0,"{'numero': '202018', 'objeto': 'Objeto: Serviç..."
4,40963966,"{'numero': '000032017', 'objeto': 'Objeto: Pre...",None,None,"{'codigo': '440001', 'nome': 'SUBSECRET.DE PLA...",None,21/05/2018,23/05/2018,21/05/2018,20/05/2019,"{'numeroInscricaoSocial': '', 'nome': 'TECNO -...",393752.0,393752.0,"{'numero': '82018', 'objeto': 'Objeto: Contrat..."


Mas... o que é esse formato estranho? 

In [16]:
df_contratos.dimCompra[0]

{'contatoResponsavel': 'PAULO SERGIO ALBERTO',
 'numero': '000112018',
 'numeroProcesso': '12804100067201812',
 'objeto': 'Objeto: Pregão Eletrônico -  Aquisição de ESTAÇÕES DE TRABALHO (DESKTOPS), conforme condições, quantidades e exigências estabelecidas no Edital e seus anexos.'}

Nosso tempo acabou... é assunto para outro capítulo!

## 4. Mais referências

* **Documentação técnica** e bastante detalhada sobre o orçamento federal é frequentemente atualizada pelo governo 
 * Acesse o [Manual Técnico do Orçamento Federal](https://www1.siop.planejamento.gov.br/mto/lib/exe/fetch.php/mto2020:mto2020-versao5.pdf) - MTO 2020

* **A Prefeitura de São Paulo** também tem uma API de Despesas e Contratos - e, neste caso, os dados não estão disponíveis para download. Para conhecer detalhes da execução orçamentária, a única maneira, por ora, é usar a API. 

 * Acesse a [API do SOF](https://api.prodam.sp.gov.br/store/apis/info?name=sof&version=v2.1.0&provider=admin)
 * [Publicação da Secretaria de Inovação e Tecnologia da PMSP ](https://copicola.prefeitura.sp.gov.br/docs/8_guia_dadosabertos.pdf)que sistematiza a história de construção da API
 * [Tutorial que fiz ](https://github.com/campagnucci/api_sof) para download de dados na API do SOF. 

* **Material do INESC** que explica o orçamento e sua relação com direitos humanos

 * Faça download da [Cartilha Orçamento e Direitos](https://www.inesc.org.br/orcamento-direitos-2018-2/) (2018) 